What the new anchor patterns may look like. 

Note this is using features of blaze

In [1]:
from blaze import Data, join, odo
import pandas as pd
import numpy as np

def get_symbol(symbol, variable, id_name='document_id'):
    'pure convenience for readable join'
    return symbol[symbol.variable == variable].relabel(value=variable)[[id_name, variable]]

def pivot(symbol, index, columns, aggfunc=np.size):
    r = odo(symbol, pd.DataFrame).pivot_table(
        index=index, columns=columns, aggfunc=aggfunc).fillna(0)
    return r

def drop_top_column_index(dataframe):
    dataframe.columns = dataframe.columns.droplevel(0)
    return dataframe

Say we have data like this:

In [2]:
df = pd.read_clipboard()
df

,document_id,variable,value
0,1,hashtag,#Foo
1,1,hashtag,#Foos
2,1,mention,Bar
3,1,community,1
4,2,hashtag,#fu
5,2,hashtag,#tagged
6,2,mention,Alberto
7,2,community,2
8,3,hashtag,#tagged
9,3,hashtag,#Foo


In [3]:
# read into blaze 
d = Data(df)

# Get handle on variables for summary
community = get_symbol(d, 'community')
hashtags = get_symbol(d, 'hashtag')
mentions = get_symbol(d, 'mention')


In [4]:
q = join(community, hashtags, how='inner')
pivot(q, 'community', 'hashtag')


document_id                  
hashtag          #Foo #Foos #fu #tagged
community                              
1                   1     1   0       0
2                   1     0   1       2

In [5]:
q = join(community, mentions, how='inner')
pivot(q, 'community', 'mention')


document_id          
mention       Alberto Bar Homer
community                      
1                   0   1     0
2                   1   0     1